[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/MagicAnimate-colab/blob/main/DensePose_colab.ipynb)

In [ ]:
%cd /content
!wget https://huggingface.co/camenduru/densepose/raw/main/Base-DensePose-RCNN-FPN.yaml -O /content/Base-DensePose-RCNN-FPN.yaml
!wget https://huggingface.co/camenduru/densepose/raw/main/densepose_rcnn_R_50_FPN_s1x.yaml -O /content/densepose_rcnn_R_50_FPN_s1x.yaml
!pip install -q gradio 
!pip install -q https://huggingface.co/camenduru/densepose/resolve/main/detectron2-0.6-cp310-cp310-linux_x86_64.whl?download=true
!pip install -q https://huggingface.co/camenduru/densepose/resolve/main/detectron2_densepose-0.6-py3-none-any.whl?download=true

import gradio as gr
from detectron2.config import get_cfg
import torch
import cv2
import numpy as np
from detectron2.engine import DefaultPredictor
from densepose import add_densepose_config
from densepose.vis.extractor import DensePoseResultExtractor
from densepose.vis.densepose_results import DensePoseResultsFineSegmentationVisualizer as Visualizer
import tempfile
import shutil

def process_video(input_video_path):
    output_video_path = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False).name
    cfg = get_cfg()
    add_densepose_config(cfg)
    cfg.merge_from_file("/content/densepose_rcnn_R_50_FPN_s1x.yaml")
    predictor = DefaultPredictor(cfg)
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        with torch.no_grad():
            outputs = predictor(frame)['instances']
        results = DensePoseResultExtractor()(outputs)
        cmap = cv2.COLORMAP_VIRIDIS
        arr = cv2.applyColorMap(np.zeros((height, width), dtype=np.uint8), cmap)
        out_frame = Visualizer(alpha=1, cmap=cmap).visualize(arr, results)        
        out.write(out_frame)
    cap.release()
    out.release()
    return output_video_path
iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="Input Video"),
    outputs=gr.Video(label="Output DensePose Video"),
    title="Video 2 DensePose"
)
iface.launch(share=True, inline=False, debug=True)